In [ ]:
started at 01:25

1. Существует массив Z = [1,2,3,4,5,6,7,8,9,10,11,12,13,14],
как из него создать массив R = [[1,2,3,4], [2,3,4,5], [3,4,5,6], ...,[11,12,13,14]]?


In [38]:
import numpy as np
from scipy import optimize

z=[1,2,3,4,5,6,7,8,9,10,11,12,13,14]
R=[]
#R.append([1,2,3,4])
#R.append([2,3,4,5])
#R=np.array(z)
#R.reshape(10,4)

In [2]:
for i in range(len(z)-3):
    #print(z[i])
    R.append([z[i], z[i+1], z[i+2], z[i+3]])

In [3]:
print(R)

[[1, 2, 3, 4], [2, 3, 4, 5], [3, 4, 5, 6], [4, 5, 6, 7], [5, 6, 7, 8], [6, 7, 8, 9], [7, 8, 9, 10], [8, 9, 10, 11], [9, 10, 11, 12], [10, 11, 12, 13], [11, 12, 13, 14]]


continued at 01:48

2. Дана произвольная матрица A. Рассчитать ранг матрицы A.

In [4]:
A=np.matrix([[1,2,5,6,17,1],\
  [0,0, 0,3,9,4],\
               [0,16, 7,3,11,0],\
  [0,0,2,6,1,13]])

In [5]:
#Можно и полезно было бы сделать через цикл и линейные преобразования над строками и столбцами                      #print(A) #(k,l)=A.shape  A.sort(axis=1) #print(k,l) # A.item(4)
# однако из-за нехватки времени, воспользуемся встроенной функцией
np.linalg.matrix_rank(A)

4

continued at 02:10
paused at 3:02

3. Найти лучший алгоритм для решения следующей задачи:
min (x1x4(x1+x2+x3)+x3).
Условия:
x1x2x3x4≥30
x1^2+x2^2+x3^2+x4^2=60
2≤x1,x2,x3,x4≤6
Начальное приближение:
x=(1,5,5,1)

In [186]:
def f(x):
    return x[0]*x[3]*(x[0]+x[1]+x[2])+x[2]

In [187]:
from scipy.optimize import Bounds
bounds = Bounds([2, 2, 2, 2], [6, 6, 6, 6])

In [188]:
#f([1,5,5,1]) #16         #f([2,2,2,2]) #26     #f([6,6,6,6]) #654  #f([3.872983346207417,3.872983346207417,3.872983346207417,3.872983346207417])  
x0=[1,5,5,1]

In [189]:
#Option CORRECT!
cons = ({'type': 'ineq', 'fun': lambda x:  x[0]*x[1]*x[2]*x[3]-30.0},
         {'type': 'eq', 'fun': lambda x: x[0]**2 + x[1]**2 +x[2]**2+x[3]**2-60})

In [190]:
optimize.minimize(f, x0, method='SLSQP', bounds=bounds,constraints=cons)

     fun: 52.00000009701402
     jac: array([28.,  4.,  5., 24.])
 message: 'Optimization terminated successfully.'
    nfev: 30
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([2.        , 6.        , 4.00000002, 2.        ])

In [ ]:
# Это точное решение до 8-го знака всего за 5 итераций. 
# Ф-ия на заданном интервале возрастает по всем аргументам. 
# Быстрее всего она растет по х1, х4. Значит для минимизации они должны быть равны 2. 
# Медленнее всего функция растет по х2, значит он масксимальный на сфере и в заданном интервале т.е. равен 6
# при этом получается х3 на сфере равен 4.


In [204]:
#option 10 for COBYLA. 'Cobyla' dont support 'eq' constraints. 
#So I replaced it the original condition x1^2+x2^2+x3^2+x4^2=60 with two 'ineq' constraints and parameter epsilon
#x1^2+x2^2+x3^2+x4^2-60-epsilon>0 and  60-x1^2-x2^2-x3^2-x4^2+epsilon>0
epsilon=0.0000000000015
cons = ({'type': 'ineq', 'fun': lambda x:  x[0]*x[1]*x[2]*x[3]-30.0},
         {'type': 'ineq', 'fun': lambda x: x[0]**2 + x[1]**2 +x[2]**2+x[3]**2-60+epsilon},
        {'type': 'ineq', 'fun': lambda x: epsilon+60-x[0]**2 - x[1]**2 -x[2]**2-x[3]**2},
       {'type': 'ineq', 'fun': lambda x: x[0]-2},
       {'type': 'ineq', 'fun': lambda x: x[1]-2},
       {'type': 'ineq', 'fun': lambda x: x[2]-2},
       {'type': 'ineq', 'fun': lambda x: x[3]-2},
       {'type': 'ineq', 'fun': lambda x: 6-x[0]},
       {'type': 'ineq', 'fun': lambda x: 6-x[1]},
       {'type': 'ineq', 'fun': lambda x: 6-x[2]},
       {'type': 'ineq', 'fun': lambda x: 6-x[3]})

In [205]:
optimize.minimize(f, x0, method='COBYLA', \
                      constraints=cons)#, bounds=bounds, )   print(4+4.9521735**2+5.24176284**2+4)

     fun: 51.99999998085571
   maxcv: 3.062935785015573e-08
 message: 'Optimization terminated successfully.'
    nfev: 161
  status: 1
 success: True
       x: array([2., 6., 4., 2.])

In [ ]:
# Алгоритм отработал. Ошибка в 8-м знаке. Итераций 161. 

In [ ]:
# Согласно документации scipy и найденным примерам 
# нелинейные ограничения поддерживает приведенные выше два метода и ‘trust-constr’

In [ ]:
#Реализация ‘trust-constr’

In [193]:
from scipy.optimize import LinearConstraint
linear_constraint = LinearConstraint(np.eye(4), [2, 2, 2, 2], [6, 6, 6, 6])


In [206]:
#NonlinearConstraint
#Option I

#epsilon=1.25
epsilon=0.000000015

def cons_f(x):
    #return [x[0]**2 + x[1]**2 +x[2]**2+x[3]**2-60+epsilon, 60-x[0]**2 - x[1]**2 -x[2]**2-x[3]**2-epsilon, 30-x[0]*x[1]*x[2]*[3]]
    return [ x[0]**2+x[1]**2+x[2]**2+x[3]**2-60-epsilon  , \
                60-epsilon-x[0]**2-x[1]**2-x[2]**2-x[3]**2  , \
                30-x[0]*x[1]*x[2]*x[3] ]
    #return [x[0],x[1],x[2], x[3]]

def cons_J(x):
    return [[2*x[0], 2*x[1], 2*x[2], 2*x[3]],    \
                [-2*x[0], -2*x[1], -2*x[2], -2*x[3]],   \
                [x[1]*x[2]*x[3], x[0]*x[2]*x[3], x[0]*x[1]*x[3], x[0]*x[1]*x[2]]]

#option 1
def cons_H(x, v):
    return v[0]*np.array([[2, 2,2,2], [-2, -2,-2,-2], [0,0,0,0]]) + \
    v[1]*np.array([[2, 2,2,2], [-2, -2,-2,-2], [0,0,0,0]])+\
    v[2]*np.array([[2, 2,2,2], [-2, -2,-2,-2], [0,0,0,0]])

#заменил условие x1^2+x2^2+x3^2+x4^2=60 на два x1^2+x2^2+x3^2+x4^2-60<epsilon -x1^2-x2^2-x3^2-x4^2+60>-epsilon 



from scipy.optimize import NonlinearConstraint
min_nlc=[-np.inf, -np.inf, -np.inf]
max_nlc=[0, 0, 0]
#max_nlc=[-epsilon, epsilon, 30]
#nonlinear_constraint = NonlinearConstraint(cons_f, min_nlc, max_nlc, jac=cons_J, hess=cons_H)
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 0, jac=cons_J, hess=cons_H)


#option 2
from scipy.sparse import csc_matrix
def cons_H_sparse(x, v):
    return v[0]*csc_matrix([[2, 2,2,2], [-2, -2,-2,-2], [0,0,0,0]]) \
    + v[1]*csc_matrix([[2, 2,2,2], [-2, -2,-2,-2], [0,0,0,0]]) + \
    v[2]*[[2, 2,2,2], [-2, -2,-2,-2], [0,0,0,0]]
# nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 0, jac=cons_J, hess=cons_H_sparse)


In [ ]:
#далее использовал встроенные якобиан и гессиан для ограничений.   
#вычисленные мной cons_J и cons_H, похоже неверны. Даже по размерности

In [208]:
from scipy.optimize import BFGS
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 0, jac=cons_J, hess=BFGS())

In [209]:
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 0, jac=cons_J, hess='2-point')

In [210]:
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 0, jac='2-point', hess=BFGS())

In [160]:
#min (x1x4(x1+x2+x3)+x3)
def jacobian(x):
    return np.array((2*x[0]*x[3]+x[1]*x[3]+x[3]*x[2],x[0]*x[3], x[0]*x[3]+1, x[0]*(x[0]+x[1]+x[2]))) 

In [161]:
def f_hess(x):
    H=np.matrix([[4*x[3], x[3], x[3], 2*x[0]+x[1]+x[2]],
                 [x[3],          0 ,   0,          x[0]],
                 [x[3],          0 ,   0,          x[0]],
                 [2*x[0]+x[1]+x[2], x[0], x[0], 0]])
    return H

In [211]:
optimize.minimize(f, x0, method='trust-constr', jac=jacobian, hess=f_hess, \
                      constraints=nonlinear_constraint, bounds=bounds)  

#constraints=[linear_constraint, nonlinear_constraint], )

 barrier_parameter: 1.0240000000000006e-08
 barrier_tolerance: 1.0240000000000006e-08
          cg_niter: 69
      cg_stop_cond: 4
            constr: [array([-2.40876426e-08, -5.91235771e-09, -6.59999994e+01]), array([1.99999999, 6.        , 4.00000001, 1.99999999])]
       constr_nfev: [290, 0]
       constr_nhev: [0, 0]
       constr_njev: [0, 0]
    constr_penalty: 23.48175858197805
  constr_violation: 8.63964655373195e-09
    execution_time: 0.3870060443878174
               fun: 51.999999592086716
              grad: array([27.99999986,  3.99999997,  4.99999997, 23.99999989])
               jac: [array([[  4.00000003,  12.00000008,   8.00000012,   4.00000003],
       [ -4.00000003, -12.00000008,  -8.00000006,  -4.00000003],
       [-48.00000036, -16.        , -24.        , -47.99999988]]), array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])]
   lagrangian_grad: array([-7.10542736e-15,  3.10862447e-15, -1.40148201e-15,  3.55271368e-

In [ ]:
#Ошибка в 7 знаке. Итераций как я понял больше 100 

In [ ]:
#На этом решение заканчивается. Лучший метод для этой задачи - "SLSQP"

‘trust-constr’ (somehow)

Constraints definition (only for COBYLA and SLSQP)

‘SLSQP’ (success)
‘COBYLA’ (success low accuracy)


Bounds for variables (only for L-BFGS-B, TNC and SLSQP).

‘L-BFGS-B’
‘TNC’ 

optimize.minimize(f, x0, method='TNC', \
                      bounds=bounds)


‘BFGS’


‘Powell’
‘CG’
‘Newton-CG’
‘dogleg’
‘trust-ncg’
‘trust-exact’ 
‘trust-krylov’

Dont support constraints
‘Nelder-Mead’ 
